In [3]:
#Import des librairies

import fastf1 as ff1
from fastf1 import plotting
from fastf1 import utils
ff1.Cache.enable_cache('D:\Data analyst\Jedha\Projet\Projet F1\doc_cache')
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
from plotly.subplots import make_subplots
pio.templates.default = "plotly_dark"
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import requests

import pandas as pd

# Définition des variables

# Course ??
tourTot = 78
perteArret = 25


#Génère le nombre de tour avec 2 arrêts

liste =[]
df_combi = pd.DataFrame(columns=['Train1', 'Train2', 'Train3'])

for i in range(1,tourTot-1):
    T1 = i
    
    
    for j in range(1, tourTot-1):
        
        if (i+j) > tourTot-1:
            continue
        T2 = j
        T3 = tourTot - j - i
        df_combi = df_combi.append({'Train1' : T1, 'Train2' : T2, 'Train3' : T3}, ignore_index = True)
        
        
#Génère le nombre de tour avec 1 arrêt

liste =[]
df_combi2 = pd.DataFrame(columns=['Train1', 'Train2'])

for i in range(1,tourTot):

    T1 = i
    T2 = tourTot-i
    df_combi2 = df_combi2.append({'Train1' : T1, 'Train2' : T2}, ignore_index = True)
        

# Ouvre le csv avec le temps par tour et type de pneu
dffp_complet = pd.read_csv('D:\Data analyst\Jedha\Projet\Projet F1\Data\dffp_complet.csv', sep = ";")
dffp_complet.head()


# Mets les temps sous forme de tableau

dffp_soft = dffp_complet[dffp_complet['Compound']=='SOFT']
dffp_soft.drop(columns = ['Compound'], inplace = True)
dffp_soft.rename(columns={'LapTime': 'S'}, inplace = True)

dffp_medium = dffp_complet[dffp_complet['Compound']=='MEDIUM']
dffp_medium.drop(columns = ['Compound'], inplace = True)
dffp_medium.rename(columns={'LapTime': 'M'}, inplace = True)

dffp_hard = dffp_complet[dffp_complet['Compound']=='HARD']
dffp_hard.drop(columns = ['Compound'], inplace = True)
dffp_hard.rename(columns={'LapTime': 'H'}, inplace = True)

dffp_final = dffp_soft.merge(dffp_medium, how = 'inner', left_on = 'tyreLife', right_on = 'tyreLife' )
dffp_final = dffp_final.merge(dffp_hard, how = 'inner', left_on = 'tyreLife', right_on = 'tyreLife' )


#Ouvre le csv avec les combinaisons pour 1 arrêts
tyreCombi2 = pd.read_csv('D:\Data analyst\Jedha\Projet\Projet F1\Data\TyreCombi2.csv', sep = ";")


#Ouvre le csv avec les combinaisons pour 2 arrêts
tyreCombi3 = pd.read_csv('D:\Data analyst\Jedha\Projet\Projet F1\Data\TyreCombi3.csv', sep = ";")



# Boucle sur tous les types de pneu pour 2 arrêts afin de générer tous les temps totaux pour la course

time1 = 0
time2 = 0
time3 = 0
listeT = []
liste_combi = []
df_timerace3 = pd.DataFrame(columns=['combiTyre','Train1', 'Train2', 'Train3','TimeT1', 'TimeT2', 'TimeT3', 'Total_time'])
df_timeracebis = pd.DataFrame(columns=['combiTyre','Train1', 'Train2', 'Train3','TimeT1', 'TimeT2', 'TimeT3', 'Total_time'])
df_timeraceter = pd.DataFrame(columns=['combiTyre','Train1', 'Train2', 'Train3','TimeT1', 'TimeT2', 'TimeT3', 'Total_time'])
df_timerace3= df_timeraceter

#Boucle sur le df tyre_combi

for a in range(0,len(tyreCombi3)):
    lig = a
    df_timeracebis = df_timeraceter

    # Calcul du T1 en fonction du type de pneu 
    col = 0
    liste_combi = []
    liste_combi.append(tyreCombi3.iloc[lig,col])
    
    if tyreCombi3.iloc[lig,col] == 'S':
        colT = 1
    elif tyreCombi3.iloc[lig,col] == 'M':
        colT = 2
    else :
        colT = 3
         
    #Boucle sur la longueur du df
    for l in range(len(df_combi)):
        #nombre de tour égal à la valeur de la 1ère colonne
        nbTour = df_combi.iloc[l,0]
            
        #Addition des nbTours premières lignes
            
        for i in range(nbTour): #J'itère autant de fois qu'il y a de tours
            time1 = time1 + dffp_final.iloc[i,colT]      
            
        df_timeracebis = df_timeracebis.append({'TimeT1' : time1}, ignore_index = True)
        df_timeracebis.iloc[l,1] = nbTour
        time1 = 0
        
        
# Calcul du T2 en fonction du type de pneu          
        
    col = 1
    liste_combi.append(tyreCombi3.iloc[lig,col])
    
    if tyreCombi3.iloc[lig,col] == 'S':
        colT = 1
    elif tyreCombi3.iloc[lig,col] == 'M':
        colT = 2
    else :
        colT = 3
        
        
    #Boucle sur la longueur du df    
    for l in range(len(df_combi)):
        #nombre de tour égal à la valeur de la 1ère colonne
        nbTour = df_combi.iloc[l,1]
        
        #Addition des nbTours premières lignes
        
        for i in range(nbTour): #J'itère autant de fois qu'il y a de tours
            time2 = time2 + dffp_final.iloc[i,colT]
            
        df_timeracebis.iloc[l,5]=time2
        df_timeracebis.iloc[l,2] = nbTour
        time2 = 0
        
        
# Calcul du T3 en fonction du type de pneu    
    
    col = 2
    liste_combi.append(tyreCombi3.iloc[lig,col])
    
    if tyreCombi3.iloc[lig,col] == 'S':
        colT = 1
    elif tyreCombi3.iloc[lig,col] == 'M':
        colT = 2
    else :
        colT = 3
        
    
    #Boucle sur la longueur du df    
    for l in range(len(df_combi)):
        #nombre de tour égal à la valeur de la 1ère colonne
        nbTour = df_combi.iloc[l,2]
        
        #Addition des nbTours premières lignes
        
        for i in range(nbTour): #J'itère autant de fois qu'il y a de tours
            time3 = time3 + dffp_final.iloc[i,colT]
            
        df_timeracebis.iloc[l,6]=time3
        df_timeracebis.iloc[l,3] = nbTour
        time3 = 0
        
        df_timeracebis.iloc[l,0] = "".join(liste_combi)

    df_timerace3 = pd.concat([df_timerace3,df_timeracebis], ignore_index = True)

#Calcul du temps total
df_timerace3['Total_time'] = df_timerace3['TimeT1'] + df_timerace3['TimeT2'] + df_timerace3['TimeT3'] + ( 2 * perteArret)


# Boucle sur tous les types de pneu pour 2 arrêts afin de générer tous les temps totaux pour la course

time1 = 0
time2 = 0
time3 = 0
listeT = []
liste_combi = []
df_timerace2 = pd.DataFrame(columns=['combiTyre','Train1', 'Train2', 'Train3','TimeT1', 'TimeT2', 'TimeT3', 'Total_time'])
df_timeracebis = pd.DataFrame(columns=['combiTyre','Train1', 'Train2', 'Train3','TimeT1', 'TimeT2', 'TimeT3', 'Total_time'])
df_timeracequa = pd.DataFrame(columns=['combiTyre','Train1', 'Train2', 'Train3','TimeT1', 'TimeT2', 'TimeT3', 'Total_time'])
df_timerace2 = df_timeracequa

#Boucle sur le df tyre_combi

for a in range(0,len(tyreCombi2)):
    lig = a
    df_timeracebis = df_timeraceter

    # Calcul du T1 en fonction du type de pneu 
    col = 0
    liste_combi = []
    liste_combi.append(tyreCombi2.iloc[lig,col])
    
    if tyreCombi2.iloc[lig,col] == 'S':
        colT = 1
    elif tyreCombi2.iloc[lig,col] == 'M':
        colT = 2
    else :
        colT = 3
         
    #Boucle sur la longueur du df
    for l in range(len(df_combi2)):
        #nombre de tour égal à la valeur de la 1ère colonne
        nbTour = df_combi2.iloc[l,0]
            
        #Addition des nbTours premières lignes
            
        for i in range(nbTour): #J'itère autant de fois qu'il y a de tours
            time1 = time1 + dffp_final.iloc[i,colT]      
            
        df_timeracebis = df_timeracebis.append({'TimeT1' : time1}, ignore_index = True)
        df_timeracebis.iloc[l,1] = nbTour
        time1 = 0
        
        
# Calcul du T2 en fonction du type de pneu          
        
    col = 1
    liste_combi.append(tyreCombi2.iloc[lig,col])
    
    if tyreCombi2.iloc[lig,col] == 'S':
        colT = 1
    elif tyreCombi2.iloc[lig,col] == 'M':
        colT = 2
    else :
        colT = 3
        
        
    #Boucle sur la longueur du df    
    for l in range(len(df_combi2)):
        #nombre de tour égal à la valeur de la 2ème colonne
        nbTour = df_combi2.iloc[l,1]
        
        #Addition des nbTours premières lignes
        
        for i in range(nbTour): #J'itère autant de fois qu'il y a de tours
            time2 = time2 + dffp_final.iloc[i,colT]
            
        df_timeracebis.iloc[l,5]=time2
        df_timeracebis.iloc[l,2] = nbTour
        time2 = 0
        
        
# Génération du Train3 et Time3 à 0    
  
    
    #Boucle sur la longueur du df    
    for l in range(len(df_combi2)):        
            
        df_timeracebis.iloc[l,6]=0
        df_timeracebis.iloc[l,3] = 0
        
        
        df_timeracebis.iloc[l,0] = "".join(liste_combi)

    df_timerace2 = pd.concat([df_timerace2,df_timeracebis], ignore_index = True)

#Calcul du temsp total
df_timerace2['Total_time'] = df_timerace2['TimeT1'] + df_timerace2['TimeT2'] + df_timerace2['TimeT3']  + ( perteArret)


#Concatène les 2 df
df_timerace = pd.concat([df_timerace2,df_timerace3], ignore_index = True)

# Affiche les 10 combinaisons les plus performantes
df_timerace = df_timerace.sort_values(['Total_time'], ascending=True)
df_timerace.head(10)

c:\Users\clefe\anaconda3\lib\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
c:\Users\clefe\anaconda3\lib\site-packages\pandas\core\frame.py:5039: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


,combiTyre,Train1,Train2,Train3,TimeT1,TimeT2,TimeT3,Total_time
272,MH,42.0,36.0,0.0,3121.154,2659.651,0.0,5805.805
273,MH,43.0,35.0,0.0,3195.681,2585.630,0.0,5806.311
276,MH,46.0,32.0,0.0,3416.742,2364.803,0.0,5806.545
262,MH,32.0,46.0,0.0,2373.230,3408.430,0.0,5806.660
271,MH,41.0,37.0,0.0,3046.346,2735.462,0.0,5806.808
274,MH,44.0,34.0,0.0,3269.653,2512.281,0.0,5806.934
275,MH,45.0,33.0,0.0,3343.028,2439.071,0.0,5807.099
261,MH,31.0,47.0,0.0,2299.625,3482.645,0.0,5807.270
270,MH,40.0,38.0,0.0,2970.727,2811.552,0.0,5807.279
264,MH,34.0,44.0,0.0,2523.726,3259.240,0.0,5807.966


In [10]:
# Sélection des 10 meilleurs combinaisons
df_timerace = df_timerace.sort_values(['Total_time'], ascending=True)
df_timerace.head(10)

,combiTyre,Train1,Train2,Train3,TimeT1,TimeT2,TimeT3,Total_time
272,MH,42.0,36.0,0.0,3121.154,2659.651,0.0,5805.805
273,MH,43.0,35.0,0.0,3195.681,2585.630,0.0,5806.311
276,MH,46.0,32.0,0.0,3416.742,2364.803,0.0,5806.545
262,MH,32.0,46.0,0.0,2373.230,3408.430,0.0,5806.660
271,MH,41.0,37.0,0.0,3046.346,2735.462,0.0,5806.808
274,MH,44.0,34.0,0.0,3269.653,2512.281,0.0,5806.934
275,MH,45.0,33.0,0.0,3343.028,2439.071,0.0,5807.099
261,MH,31.0,47.0,0.0,2299.625,3482.645,0.0,5807.270
270,MH,40.0,38.0,0.0,2970.727,2811.552,0.0,5807.279
264,MH,34.0,44.0,0.0,2523.726,3259.240,0.0,5807.966


In [184]:
df_final = pd.DataFrame(columns=['combiTyre','Train1', 'Train2', 'Train3','TimeT1', 'TimeT2', 'TimeT3', 'Total_time', 'Total_tour'])

df_final = df_final.append(df_timerace.iloc[0,:])


In [187]:
df_final

,combiTyre,Train1,Train2,Train3,TimeT1,TimeT2,TimeT3,Total_time,Total_tour
272,MH,42.0,36.0,0.0,3121.154,2659.651,0.0,5805.805,78.0
276,MH,46.0,32.0,0.0,3416.742,2364.803,0.0,5806.545,78.0
262,MH,32.0,46.0,0.0,2373.230,3408.430,0.0,5806.660,78.0


In [186]:
# Crée un dataframe avec les 3 meilleures temps présentant un écart significatif avec le 1er

#Variables définissant les écarts de tours à prendre en compte sur le train 1
ecart1 = 3
ecart2 = 3
nbMax = 3


i=0

for t in range(1,len(df_timerace)):
    if (len(df_timerace.iloc[t,0]) != len(df_timerace.iloc[0,0])) or (df_timerace.iloc[t,1] > df_timerace.iloc[0,1] + ecart1) or (df_timerace.iloc[t,1] < df_timerace.iloc[0,1] - ecart2) :
        df_final = df_final.append(df_timerace.iloc[t,:])
        i += 1
    if i == nbMax - 1:
        break
    
df_final['Total_tour'] = df_final['Train1'] + df_final['Train2'] +df_final['Train3']

In [203]:
df_viz = pd.DataFrame(columns=['combiTyre','tyreType', 'Tour', 'Time', 'Total_time'])
df_viz = df_viz.append({'combiTyre' : df_final.iloc[0,0], 'tyreType' : df_final.iloc[0,0][0], 'Tour' : 1, 'tyreTourTot' : df_final.iloc[0,1], 'Time' : 0, 'Total_time' : 0,  'Total_tour' : df_final.iloc[0,8]}, ignore_index = True)


In [204]:
df_viz

,combiTyre,tyreType,Tour,tyreTourTot,Time,Total_time,Total_tour
0,MH,M,1,42.0,0,0,78.0


In [205]:
for z in range(1, int(df_viz.iloc[0,3])) :
    df_viz.loc[z,['combiTyre']] = df_viz.loc[z-1,['combiTyre']]
    df_viz.loc[z,['tyreType']] = df_viz.loc[z-1,['tyreType']]
    df_viz.loc[z,['Tour']] = df_viz.loc[z-1,['Tour']] + 1
    df_viz.loc[z,['tyreTourTot']] = df_viz.loc[z-1,['tyreTourTot']]
    df_viz.loc[z,['Total_tour']] = df_viz.loc[z-1,['Total_tour']]

In [ ]:
for y in ([42, 36]) :
    
    for z in range(1, int(df_viz.iloc[0,3])) :
        df_viz.loc[z,['combiTyre']] = df_viz.loc[z-1,['combiTyre']]
        df_viz.loc[z,['tyreType']] = df_viz.loc[z-1,['tyreType']]
        df_viz.loc[z,['Tour']] = df_viz.loc[z-1,['Tour']] + 1
        df_viz.loc[z,['tyreTourTot']] = df_viz.loc[z-1,['tyreTourTot']]
        df_viz.loc[z,['Total_tour']] = df_viz.loc[z-1,['Total_tour']]
        
    df_viz = df_viz.append({'combiTyre' : df_final.iloc[0,0], 'tyreType' : df_final.iloc[0,0][1], 'Tour' : 1, 'tyreTourTot' : df_final.iloc[y,1], 'Time' : 0, 'Total_time' : 0,  'Total_tour' : df_final.iloc[0,8]}, ignore_index = True)

In [218]:
dffp_complet = pd.read_csv('D:\Data analyst\Jedha\Projet\Projet F1\Data\dffp_complet.csv', sep = ";")


# Mets les temps sous forme de tableau

dffp_soft = dffp_complet[dffp_complet['Compound']=='SOFT']
dffp_soft.drop(columns = ['Compound'], inplace = True)
dffp_soft.rename(columns={'LapTime': 'S'}, inplace = True)

dffp_medium = dffp_complet[dffp_complet['Compound']=='MEDIUM']
dffp_medium.drop(columns = ['Compound'], inplace = True)
dffp_medium.rename(columns={'LapTime': 'M'}, inplace = True)

dffp_hard = dffp_complet[dffp_complet['Compound']=='HARD']
dffp_hard.drop(columns = ['Compound'], inplace = True)
dffp_hard.rename(columns={'LapTime': 'H'}, inplace = True)

dffp_final = dffp_soft.merge(dffp_medium, how = 'inner', left_on = 'tyreLife', right_on = 'tyreLife' )
dffp_final = dffp_final.merge(dffp_hard, how = 'inner', left_on = 'tyreLife', right_on = 'tyreLife' )

dffp_final.head(15)

c:\Users\clefe\anaconda3\lib\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
c:\Users\clefe\anaconda3\lib\site-packages\pandas\core\frame.py:5039: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


,tyreLife,S,M,H
0,1,76.452,74.888,74.146
1,2,76.177,74.059,74.639
2,3,76.110,74.052,74.008
3,4,76.140,73.294,73.647
4,5,76.860,73.884,73.434
5,6,75.539,74.271,73.297
6,7,77.558,73.406,72.885
7,8,79.670,73.662,73.068
8,9,79.171,77.580,72.702
9,10,78.457,77.316,72.517


In [219]:
df_final

,combiTyre,Train1,Train2,Train3,TimeT1,TimeT2,TimeT3,Total_time,Total_tour
272,MH,42.0,36.0,0.0,3121.154,2659.651,0.0,5805.805,78.0
276,MH,46.0,32.0,0.0,3416.742,2364.803,0.0,5806.545,78.0
262,MH,32.0,46.0,0.0,2373.230,3408.430,0.0,5806.660,78.0


In [270]:
dffp_final.iloc[0,1]

76.452

In [309]:

    if tyreType == "S":
        col = 1
    elif tyreType == "M":
        col = 2
    else :
        col = 3
        
    ligneMax =  int(df_final.iloc[0,1])
     
    maListeT = dffp_final.iloc[0:ligneMax, col].tolist()
    

In [421]:
df_viz = pd.DataFrame(columns=['combiTyre','tyreType', 'Tour', 'Time', 'Total_time'])



listCombi = list(df_final.iloc[0,0].strip())
cpt = 1
maListe = []
maListeTypeT = []
ligneMax = 0

for tyreType in listCombi:
    if tyreType == "S":
        col = 1
    elif tyreType == "M":
        col = 2
    else :
        col = 3
    


    ligneMax =  int(df_final.iloc[0,cpt])
    cpt += 1
    
    maListeT = dffp_final.iloc[0:ligneMax, col].tolist()
    maListe = maListe + maListeT

    maListeM = list(len(maListeT) * tyreType.strip())
    maListeTypeT = maListeTypeT + maListeM


    
df_viz['Time'] = maListe
df_viz['combiTyre'] = df_final.iloc[0,0]
df_viz['Tour'] = df_viz.index + 1
df_viz['tyreType'] = maListeTypeT

#Ajout du temps d'arrêt

arr1 = int(df_final.iloc[0,1]) - 1
timeList = df_viz['Time'].tolist()
timeList[arr1] = timeList[arr1]+25

if df_final.iloc[0,2] != 0:
    arr = int(df_final.iloc[0,2]) - 1
    timeList = df_viz['Time'].tolist()
    timeList[arr1] = timeList[arr1]+25
    
df_viz['Time2'] = timeList




,combiTyre,tyreType,Tour,Time,Total_time,Time2
39,MH,M,40,74.172,NaN,74.172
40,MH,M,41,75.619,NaN,75.619
41,MH,M,42,74.808,NaN,99.808
42,MH,H,43,74.146,NaN,74.146


In [418]:
timeList = df_viz['Time'].tolist()
timeList[41] = timeList[41]+25
timeList[35:45]
df_viz['Time2'] = timeList
df_viz.iloc[39:43,]

,combiTyre,tyreType,Tour,Time,Total_time,Time2
39,MH,M,40,74.172,NaN,74.172
40,MH,M,41,75.619,NaN,75.619
41,MH,M,42,74.808,NaN,99.808
42,MH,H,43,74.146,NaN,74.146


In [392]:
maListeM = list(len(maListeT) * 'M'.strip())


In [395]:
listCombi

['M',
 'M',
 'M',
 'M',
 'M',
 'M',
 'M',
 'M',
 'M',
 'M',
 'M',
 'M',
 'M',
 'M',
 'M',
 'M',
 'M',
 'M',
 'M',
 'M',
 'M',
 'M',
 'M',
 'M',
 'M',
 'M',
 'M',
 'M',
 'M',
 'M',
 'M',
 'M',
 'M',
 'M',
 'M',
 'M']

In [397]:
df_viz

,combiTyre,tyreType,Tour,Time,Total_time
0,MH,M,1,74.888,NaN
1,MH,M,2,74.059,NaN
2,MH,M,3,74.052,NaN
3,MH,M,4,73.294,NaN
4,MH,M,5,73.884,NaN
5,MH,M,6,74.271,NaN
6,MH,M,7,73.406,NaN
7,MH,M,8,73.662,NaN
8,MH,M,9,77.580,NaN
9,MH,M,10,77.316,NaN
